In [1]:
import tkinter as tk
from tkinter import ttk
import tkinter.messagebox as messagebox

class App:
    def __init__(self, master, size=100):
        self.master = master
        self.master.title("Hash Table")
        self.master.geometry("900x500")
        
        self.title_label = tk.Label(master, text="Hash Table ", font=("Helvetica", 16, "bold"))
        self.title_label.pack(pady=10)  # Add some padding in the y-direction

        # Entry field for entering values
        self.value_entry = tk.Entry(master)
        self.value_entry.pack(side="top", padx=10, pady=5)

        # Buttons for adding, searching, and deleting
        self.add_button = tk.Button(master, text="Add", command=self.add_operation)
        self.add_button.pack(side="top", padx=10, pady=5)

        self.search_button = tk.Button(master, text="Search", command=self.search_operation)
        self.search_button.pack(side="top", padx=10, pady=5)

        self.delete_button = tk.Button(master, text="Delete", command=self.delete_operation)
        self.delete_button.pack(side="top", padx=10, pady=5)
        
        self.hash_functions = ["hash", "hash_middle_square", "hash_transformation_radix"]
        self.selected_hash_function = tk.StringVar(master)
        self.selected_hash_function.set(self.hash_functions[0])  # Default selection
        self.hash_function_menu = tk.OptionMenu(master, self.selected_hash_function, *self.hash_functions)
        self.hash_function_menu.pack(side="top", padx=10, pady=5)

        # Create a frame to contain the hash table
        self.hash_table_frame = tk.Frame(master)
        self.hash_table_frame.pack(side="left", padx=10, pady=10, expand=True, fill="both")

        # Create a treeview for the hash table
        self.hash_table_tree = ttk.Treeview(self.hash_table_frame, columns=("Index", "Key", "Value"), show="headings")
        self.hash_table_tree.heading("Index", text="Index")
        self.hash_table_tree.heading("Key", text="Key")
        self.hash_table_tree.heading("Value", text="Value")
        self.hash_table_tree.pack(expand=True, fill="both")
        
        # Initialize the array with None values
        self.array = [None] * size

        # Update the table with initial values
        self.update_table()
        
    def add_operation(self):
        value = self.value_entry.get()
        
        # Check if the value already exists in the array
        if value in self.array:
            messagebox.showinfo("Addition Result", f"Value '{value}' already exists in the hash table.")
            return
        
        hash_function = self.selected_hash_function.get()
        if hash_function == "hash":
            index = hash(value) % len(self.array)
        elif hash_function == "hash_middle_square":
            index = self.hash_middle_square(value) % len(self.array)
        elif hash_function == "hash_transformation_radix":
            index = self.hash_transformation_radix(value) % len(self.array)
        else:
            messagebox.showerror("Error", "Invalid hash function selected.")
            return

        # Check if the calculated index is already occupied
        if self.array[index] is not None:
            # Collision handling: Linear probing
            next_index = (index + 1) % len(self.array)
            while next_index != index:
                if self.array[next_index] is None:
                    index = next_index
                    break
                next_index = (next_index + 1) % len(self.array)

        # Insert value at the calculated index
        self.array[index] = value
        self.update_table()

    def search_operation(self):
        value = self.value_entry.get()
        found_indices = []
        for index, item in enumerate(self.array):
            if item == value:
                found_indices.append(index)
        if found_indices:
            messagebox.showinfo("Search Result", f"Value '{value}' found at indices: {found_indices}")
        else:
            messagebox.showinfo("Search Result", f"Value '{value}' not found in the hash table.")

    def delete_operation(self):
        value = self.value_entry.get()
        deleted_count = 0
        for index, item in enumerate(self.array):
            if item == value:
                self.array[index] = None
                deleted_count += 1
        if deleted_count > 0:
            messagebox.showinfo("Deletion Result", f"Value '{value}' deleted {deleted_count} times from the hash table.")
            self.update_table()
        else:
            messagebox.showinfo("Deletion Result", f"Value '{value}' not found in the hash table.")

    def update_table(self):
        # Clear existing rows
        for item in self.hash_table_tree.get_children():
            self.hash_table_tree.delete(item)
        
        # Insert new rows
        for index, value in enumerate(self.array):
            if value is not None:
                self.hash_table_tree.insert("", "end", values=(index, value))
                
    def hash_middle_square(self, value):
        value = int(value)
        square = value ** 2
        str_square = str(square)
        middle = len(str_square) // 4
        hashed_value = int(str_square[middle:middle + 2])
        return hashed_value 
    
    def hash_transformation_radix(self, value):
        str_value = str(value)
        base = 10  # Define base here
        hash_value = 0

        for char in str_value:
            hash_value = (hash_value * base + int(char)) % len(self.array)

        return hash_value

def main():
    root = tk.Tk()
    app = App(root)
    root.mainloop()

if __name__ == "__main__":
    main()
